In [2]:
import os
import time

import re
import copy
import numpy as np
import pandas as pd
import h5py
import tables
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import rioxarray as rxr
import math
import pickle 
import sklearn
import graphviz
import xgboost as xgb
import lightgbm as lgbm
from pickle import dump

import contextily as cx
import rasterio
import geopandas as gpd
from shapely.geometry import Point
# from mpl_toolkits.basemap import Basemap

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from xgboost import cv
from xgboost import XGBRegressor
from xgboost import plot_importance as plot_importance_XGB
from lightgbm import LGBMRegressor
from lightgbm import plot_importance as plot_importance_LGBM

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, Activation, Dropout
from os import listdir
from os.path import isfile, join
from platform import python_version

import re

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]



print(pd.__version__) # should be 1.3.0
print(sklearn.__version__) # should be 0.24.1
print(tf.__version__) # should be 2.4.0
print(python_version()) #should be 3.8.2

1.3.5
1.0.2
2.4.1
3.9.5


In [3]:
os.getcwd()
os.chdir('..')
os.getcwd()
os.chdir('..')
os.getcwd()

'/uufs/chpc.utah.edu/common/home/civil-group1/ItalianAlps/SnowCast/Final_Model'

In [4]:

### define new regions
Region_list = ['N_Sierras',
               'S_Sierras_High',
               'S_Sierras_Low',
               'Greater_Yellowstone',
               'N_Co_Rockies',
               'SW_Mont',
               'SW_Co_Rockies',
               'GBasin',
               'N_Wasatch',
               'N_Cascade',
               'S_Wasatch',
               'SW_Mtns',
               'E_WA_N_Id_W_Mont',
               'S_Wyoming',
               'SE_Co_Rockies',
               'Sawtooth',
               'Ca_Coast',
               'E_Or',
               'N_Yellowstone',
               'S_Cascade',
               'Wa_Coast',
               'Greater_Glacier',
               'Or_Coast'
              ]

In [5]:
Model_weights_intial = {}
for Region in Region_list:
#set up filepath to extract best model
    checkpoint_filepath = 'Model/Initial_Models_Final/'+Region+ '/'
    model = checkpoint_filepath+Region+'_model.h5'
    print(model)
    model=load_model(model)
    Model_weights_intial[Region] = {}
    for i in range(1,len((model.layers))):
        weights = model.layers[i].get_weights()
        Model_weights_intial[Region][i] = weights

pickle.dump(Model_weights_intial, open("Model/Model_Calibration/Initial_MLP/Model_Weights_Intitial.pkl", "wb"))


Model/Initial_Models_Final/N_Sierras/N_Sierras_model.h5


2022-02-15 15:45:01.477077: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-15 15:45:01.478375: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-15 15:45:01.552281: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model/Initial_Models_Final/S_Sierras_High/S_Sierras_High_model.h5
Model/Initial_Models_Final/S_Sierras_Low/S_Sierras_Low_model.h5
Model/Initial_Models_Final/Greater_Yellowstone/Greater_Yellowstone_model.h5
Model/Initial_Models_Final/N_Co_Rockies/N_Co_Rockies_model.h5
Model/Initial_Models_Final/SW_Mont/SW_Mont_model.h5
Model/Initial_Models_Final/SW_Co_Rockies/SW_Co_Rockies_model.h5
Model/Initial_Models_Final/GBasin/GBasin_model.h5
Model/Initial_Models_Final/N_Wasatch/N_Wasatch_model.h5
Model/Initial_Models_Final/N_Cascade/N_Cascade_model.h5
Model/Initial_Models_Final/S_Wasatch/S_Wasatch_model.h5
Model/Initial_Models_Final/SW_Mtns/SW_Mtns_model.h5
Model/Initial_Models_Final/E_WA_N_Id_W_Mont/E_WA_N_Id_W_Mont_model.h5
Model/Initial_Models_Final/S_Wyoming/S_Wyoming_model.h5
Model/Initial_Models_Final/SE_Co_Rockies/SE_Co_Rockies_model.h5
Model/Initial_Models_Final/Sawtooth/Sawtooth_model.h5
Model/Initial_Models_Final/Ca_Coast/Ca_Coast_model.h5
Model/Initial_Models_Final/E_Or/E_Or_model.h5
Mo

In [6]:
Model_weights_final= {}
for Region in Region_list:
#set up filepath to extract best model
    checkpoint_filepath = 'Model/Prev_SWE_Models_Final/'+Region+ '/'
    model = checkpoint_filepath+Region+'_model.h5'
    print(model)
    model=load_model(model)
    Model_weights_intial[Region] = {}
    for i in range(1,len((model.layers))):
        weights = model.layers[i].get_weights()
        Model_weights_intial[Region][i] = weights

pickle.dump(Model_weights_final, open("Model/Model_Calibration/Prev_MLP/Model_Weights_Final.pkl", "wb"))


Model/Prev_SWE_Models_Final/N_Sierras/N_Sierras_model.h5
Model/Prev_SWE_Models_Final/S_Sierras_High/S_Sierras_High_model.h5
Model/Prev_SWE_Models_Final/S_Sierras_Low/S_Sierras_Low_model.h5
Model/Prev_SWE_Models_Final/Greater_Yellowstone/Greater_Yellowstone_model.h5
Model/Prev_SWE_Models_Final/N_Co_Rockies/N_Co_Rockies_model.h5
Model/Prev_SWE_Models_Final/SW_Mont/SW_Mont_model.h5
Model/Prev_SWE_Models_Final/SW_Co_Rockies/SW_Co_Rockies_model.h5
Model/Prev_SWE_Models_Final/GBasin/GBasin_model.h5
Model/Prev_SWE_Models_Final/N_Wasatch/N_Wasatch_model.h5
Model/Prev_SWE_Models_Final/N_Cascade/N_Cascade_model.h5
Model/Prev_SWE_Models_Final/S_Wasatch/S_Wasatch_model.h5
Model/Prev_SWE_Models_Final/SW_Mtns/SW_Mtns_model.h5
Model/Prev_SWE_Models_Final/E_WA_N_Id_W_Mont/E_WA_N_Id_W_Mont_model.h5
Model/Prev_SWE_Models_Final/S_Wyoming/S_Wyoming_model.h5
Model/Prev_SWE_Models_Final/SE_Co_Rockies/SE_Co_Rockies_model.h5
Model/Prev_SWE_Models_Final/Sawtooth/Sawtooth_model.h5
Model/Prev_SWE_Models_Final/Ca